In [13]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy
import re
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import json
models_folder = "../textgeneration/frontend/models/"#the folder that the model information is stored within
#eventually, change these so specific model can be received in.
character_map = "character-map.json"
model_file = "model.h5"
outputlen = 1000
temperature = 0.04 #the temperature is used to skew the probabilities in a direction, to create more/less randomness in the output.

In [7]:
# load ascii text and covert to lowercase
discordf = "../messages/discord-messages.txt"
discord = open(discordf, 'r', encoding='utf-8').read()
fbf = "../messages/facebook-messages.txt"
fb = open(fbf, 'r', encoding='utf-8').read()
essayf = "../messages/essays.txt"
essay = open(essayf, 'r', encoding='utf-8').read()

#cleanup the text a bit,
raw_text = discord.lower() + "\n" + fb.lower() + "\n" + essay.lower()
raw_text = raw_text.encode("ascii", "ignore").decode()#remove any non ascii characters.
raw_text = re.sub(r"[~#$%&*+;<=>\[\\^_\]`{|}0-9@/]","",raw_text)#strip out some ascii characters thataren't super important.

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
#save our character mapping, since we need it to actually use the model
with open(models_folder + character_map, 'w') as outfile:
    json.dump(int_to_char, outfile)
    
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers
seq_length = 300
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

Total Characters:  1529921
Total Vocab:  38
Total Patterns:  1529621


In [10]:
# define the LSTM model
model = Sequential()
model.add(LSTM(300, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

#use checkpoints to keep trac of the current progress.
filepath="checkpoints/weights-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=16, batch_size=256, callbacks=callbacks_list)
model.save(models_folder + model_file)

Epoch 1/16
5976/5976 [==============================] - ETA: 0s - loss: 2.3113
Epoch 00001: loss improved from inf to 2.31134, saving model to checkpoints\weights-01-2.3113-bigger.hdf5
5976/5976 [==============================] - 2073s 347ms/step - loss: 2.3113
Epoch 2/16
5976/5976 [==============================] - ETA: 0s - loss: 1.7863
Epoch 00002: loss improved from 2.31134 to 1.78626, saving model to checkpoints\weights-02-1.7863-bigger.hdf5
5976/5976 [==============================] - 2060s 345ms/step - loss: 1.7863
Epoch 3/16
5976/5976 [==============================] - ETA: 0s - loss: 1.6742
Epoch 00003: loss improved from 1.78626 to 1.67422, saving model to checkpoints\weights-03-1.6742-bigger.hdf5
5976/5976 [==============================] - 2061s 345ms/step - loss: 1.6742
Epoch 4/16
5976/5976 [==============================] - ETA: 0s - loss: 1.6122
Epoch 00004: loss improved from 1.67422 to 1.61223, saving model to checkpoints\weights-04-1.6122-bigger.hdf5
5976/5976 [======

In [4]:
"""model = Sequential()
model.add(LSTM(300, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.load_weights("checkpoints\weights-13-1.4986-bigger.hdf5")
model.save('model-full.h5')"""

In [8]:
# pick a random seed to use for testing the generation
start = numpy.random.randint(0, len(dataX)-1)
text = dataX[start]

In [14]:
##This code is identical to how the web code works.

#setup all the maps that will be needed for converting to and from text to the model.
with open(models_folder + character_map) as json_file:
    int_to_char = json.load(json_file)
int_to_char = { int(key) : value  for (key, value) in int_to_char.items()}#this is to fix the mapping so it has integer keys like it is supposed to
char_to_int = { value : int(key) for (key, value) in int_to_char.items()}#create a reverse map, since we'll have to conver their input.
vocab_size = len(int_to_char.keys())#the number of characters in the vocabulary


#load the lstm model from our model file.
model = tf.keras.models.load_model(models_folder + model_file)

for i in range(outputlen):
    #we convert text to be the correct shape for the lstm, and we squish all the values to be between 0 and 1
    x = numpy.reshape(text, (1, len(text), 1))
    x = x / float(vocab_size)

    #run the input through our model.
    predictions = model.predict(x, verbose=0)
    predictions = predictions / temperature #we devide the predictions by our temparature. For higher temperatures inject more randomness into the text.
    #select the prediction randomly, by sampling according to the prediction confidence.
    predicted_char = tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    result = int_to_char[predicted_char]

    #Add the character to our text, and bump out the first letter
    text.append(predicted_char)
    text = text[1:len(text)]
    print(result, end = '',flush=True)

FileNotFoundError: [Errno 2] No such file or directory: '../textgeneration/frontend/models/basic/character-map.json'